In [ ]:
#%pip install mrcfile

In [ ]:
#%pip install opencv-python

In [ ]:
#%pip install scikit-image

In [ ]:
#%pip install matplotlib

In [ ]:
#%pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
#%pip install gdown

In [ ]:
#%pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
!rm denoising
!ln -s ../../src/denoising denoising

In [ ]:
#use_OpenCV = True
#local_debug = True

In [ ]:
import logging
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#from ipywidgets import *
import cv2
import time
#import kernels
import skimage
#from skimage import io as skimage_io
import mrcfile
import logging
from collections import namedtuple
import information_theory
from denoising.volume.gaussian import Monochrome_Denoising as GF

In [ ]:
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
%%bash
ln -sf /nas/vruiz/vols/empiar11415.mrc .

In [ ]:
vol_filename = "empiar11415.mrc"

In [ ]:
Args = namedtuple("args", ["input", "output"])
fn, fe = vol_filename.split(".")
args = Args(vol_filename , fn + "_denoised." + fe)
print(args)

In [ ]:
#noisy = skimage.io.imread(args.input, plugin="tifffile").astype(np.float32)
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data

In [ ]:
Z_dim = noisy.shape[0]
Z2 = Z_dim//2

In [ ]:
plt.imshow(noisy[Z2], cmap="gray")

In [ ]:
def get_gaussian_kernel(sigma=1):
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2 :
        delta = np.zeros(number_of_coeffs)
        delta[delta.size//2] = 1
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]

std_dev = 2.0
sigma = np.array([std_dev, std_dev, std_dev])
kernel = [None]*3
kernel[0] = get_gaussian_kernel(sigma[0])
kernel[1] = get_gaussian_kernel(sigma[1])
kernel[2] = get_gaussian_kernel(sigma[2])

In [ ]:
denoiser = GF(logger)
denoised = denoiser.filter(noisy, kernel)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(denoised[Z2], cmap="gray")
axes[1].imshow(noisy[Z2], cmap="gray")
plt.show()

In [ ]:
with mrcfile.new(f"denoised.mrc", overwrite=True) as mrc:
    mrc.set_data(denoised.astype(np.float32))
    mrc.data

In [ ]:
slice_idx = noisy.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(noisy[slice_idx, :256, :256].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised[slice_idx, :256, :256].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N-Odd-Even-Registered Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()